# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
adult_dt = (pd.read_csv('../../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))


# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [5]:
# Create X that holds the features by dropping the 'income' column
X = adult_dt.drop('income', axis=1)

# Create Y that holds the target data with just the 'income' column
Y = adult_dt['income']

# Split the data into training and testing sets (70-30 split)
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, 
    test_size=0.3,    
    random_state=42
)

# Print the shapes to verify the split
print("Training set shape:", X_train.shape)
print("Testing set shape:", X_test.shape)

Training set shape: (22792, 14)
Testing set shape: (9769, 14)


## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 
+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?

*(Comment here.)*

The `random_state` in functions like `train_test_split` controls how the data is shuffled before being split into training and testing sets. By setting a specific `random_state` number, we make sure the split happens the same way every time we run the code. This is important for consistency because it allows us and others to get the same results when testing, which makes it easier to trust and verify our findings.

# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [6]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder

# Define features
numerical_features = ['age', 'fnlwgt', 'education-num', 'capital-gain', 
                     'capital-loss', 'hours-per-week']

categorical_features = ['workclass', 'education', 'marital-status', 'occupation',
                      'relationship', 'race', 'sex', 'native-country']

# Create the transformers exactly as shown in the image
num_transforms = Pipeline([
    ('knn_imputer', KNNImputer(n_neighbors=7, weights='distance')),
    ('robust_scaler', RobustScaler())
])

cat_transforms = Pipeline([
    ('simple_imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot_encoder', OneHotEncoder(handle_unknown='ignore', drop='if_binary'))
])

# Combine into ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transforms, numerical_features),
        ('cat', cat_transforms, categorical_features)
    ])

## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

# Create the full pipeline
model_pipeline = Pipeline([
    ('preprocessing', preprocessor),
    ('classifier', RandomForestClassifier())
])

# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [8]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, log_loss, roc_auc_score, accuracy_score, balanced_accuracy_score

# Define the scoring metrics
scoring = {
    'neg_log_loss': 'neg_log_loss',
    'roc_auc': 'roc_auc',
    'accuracy': 'accuracy',
    'balanced_accuracy': 'balanced_accuracy'
}

# Perform cross-validation
cv_results = cross_validate(
    model_pipeline,
    X_train,
    Y_train,
    scoring=scoring,
    cv=5,                    # 5 folds
    return_train_score=True  # To get both training and validation scores
)

# Print results
for metric in scoring.keys():
    train_scores = cv_results[f'train_{metric}']
    val_scores = cv_results[f'test_{metric}']
    
    print(f"\n{metric}:")
    print(f"Training: {train_scores.mean():.3f} (+/- {train_scores.std() * 2:.3f})")
    print(f"Validation: {val_scores.mean():.3f} (+/- {val_scores.std() * 2:.3f})")

c:\Users\marky\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\marky\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(



neg_log_loss:
Training: -0.081 (+/- 0.001)
Validation: -0.364 (+/- 0.030)

roc_auc:
Training: 1.000 (+/- 0.000)
Validation: 0.904 (+/- 0.004)

accuracy:
Training: 1.000 (+/- 0.000)
Validation: 0.853 (+/- 0.008)

balanced_accuracy:
Training: 1.000 (+/- 0.000)
Validation: 0.777 (+/- 0.007)


Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [9]:
# Create a DataFrame with fold-level results
fold_results = pd.DataFrame({
    'train_neg_log_loss': cv_results['train_neg_log_loss'],
    'test_neg_log_loss': cv_results['test_neg_log_loss'],
    'train_roc_auc': cv_results['train_roc_auc'],
    'test_roc_auc': cv_results['test_roc_auc'],
    'train_accuracy': cv_results['train_accuracy'],
    'test_accuracy': cv_results['test_accuracy'],
    'train_balanced_accuracy': cv_results['train_balanced_accuracy'],
    'test_balanced_accuracy': cv_results['test_balanced_accuracy'],
    'fold': range(5)  # Add fold numbers
})

# Sort by test (validation) neg_log_loss
fold_results_sorted = fold_results.sort_values('test_neg_log_loss', ascending=True)

# Display the sorted results
print(fold_results_sorted)

   train_neg_log_loss  test_neg_log_loss  train_roc_auc  test_roc_auc  \
2           -0.081172          -0.387352            1.0      0.902552   
4           -0.081424          -0.370726            1.0      0.904025   
0           -0.081253          -0.363663            1.0      0.904116   
3           -0.081880          -0.348868            1.0      0.907606   
1           -0.081206          -0.347085            1.0      0.902701   

   train_accuracy  test_accuracy  train_balanced_accuracy  \
2        0.999945       0.852567                 0.999887   
4        1.000000       0.858052                 1.000000   
0        1.000000       0.848432                 1.000000   
3        1.000000       0.856735                 1.000000   
1        1.000000       0.849309                 1.000000   

   test_balanced_accuracy  fold  
2                0.776849     2  
4                0.780162     4  
0                0.773344     0  
3                0.781139     3  
1                0.77177

Calculate the mean of each metric. 

In [10]:
# Calculate mean for each metric
mean_results = fold_results.drop('fold', axis=1).mean()

# Create a DataFrame with the means
mean_df = pd.DataFrame(mean_results, columns=['Mean'])

# Display the means
print("\nMean of each metric:")
print(mean_df)


Mean of each metric:
                             Mean
train_neg_log_loss      -0.081387
test_neg_log_loss       -0.363539
train_roc_auc            1.000000
test_roc_auc             0.904200
train_accuracy           0.999989
test_accuracy            0.853019
train_balanced_accuracy  0.999977
test_balanced_accuracy   0.776653


Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [11]:
# First, fit the pipeline
model_pipeline.fit(X_train, Y_train)

# Now get predictions and probability scores
Y_pred = model_pipeline.predict(X_test)
Y_pred_proba = model_pipeline.predict_proba(X_test)

# Calculate metrics
test_metrics = {
    'neg_log_loss': -log_loss(Y_test, Y_pred_proba),
    'roc_auc': roc_auc_score(Y_test, Y_pred_proba[:, 1]),
    'accuracy': accuracy_score(Y_test, Y_pred),
    'balanced_accuracy': balanced_accuracy_score(Y_test, Y_pred)
}

# Display results
print("\nTest Set Metrics:")
print(test_metrics)


Test Set Metrics:
{'neg_log_loss': -0.3889509959472722, 'roc_auc': 0.9006323739034611, 'accuracy': 0.8554611526256526, 'balanced_accuracy': 0.7765564287482312}


# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

(Answer here.)

Recoding the target variable income to a binary format (0 and 1) right after loading the data is convenient for several reasons, especially in binary classification tasks. By converting income into 0s and 1s (where 1 indicates '>50K' and 0 otherwise), we make it compatible with many machine learning algorithms and evaluation metrics in scikit-learn that expect a binary numeric format for classification. This transformation simplifies the modeling process by standardizing the target variable, ensuring compatibility with various metrics like accuracy, precision, recall, and ROC-AUC, which are specifically designed for binary outcomes.

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.